# Closures and Callables

## Closures

### Definition

Recall that functions are first-class objects in Python: e.g., they can be passed to higher-order functions such as `filter`:

In [1]:
def iseven(x):
    return x%2 == 0
list(filter(iseven, [1, 2, 3, 4, 5, 6]))

[2, 4, 6]

Side note: this is where lambdas come in handy:

In [2]:
list(filter(lambda x: x%2 == 0, [1, 2, 3, 4, 5, 6]))

[2, 4, 6]

But functions can also be _returned_ by other functions:


In [3]:
def make_squarer():
    def f(x): # nested function
        return x**2
    return f # NOTE: no parentheses!
my_squarer = make_squarer()
my_squarer(4)        

16

This is not terribly useful in and of itself. The usefulness comes from the fact that nested functions have access to their parent's scope:

In [4]:
def make_power(n): # known as a "function factory"
    def f(x):
        return x**n # n is from enclosing scope
    return f
my_power = make_power(3) # my_power is a closure
my_power(2)

8

So what happens here is that the function `f(x)` returned by `make_power` memorizes the parent scope; it carries a copy of `n` (and any other local variable in `make_power`) around with it! It is this returned function, stored in `my_power`, that's called a _closure_.

This is useful when you need to pass a function to a higher order function that expects a function with only one parameter. Lets say we want to compute the integral $$\int_{0}^5 f(x, data)\mathrm{d}x.$$ I.e., we have a function of two variables, but we want to only integrate w.r.t. $x$. 

In [5]:
# Install scipy
import sys
!{sys.executable} -m pip install scipy

In [6]:
from scipy.integrate import quad
from math import exp

def make_integrand(data):
    def f(x):
        return exp(-x*sum(data))
    return f

In [7]:
data = [1, 2, 3, 4]
my_integrand_1 = make_integrand(data)    
quad(my_integrand_1, 0, 5)[0]

0.10000000000000002

This process of turning a function of several variables into a function of a single variable is called _partial function application_. Closures are a way to achieve this.

You can think of closures the following way:
 * An _object_ is data with functions (methods) attached.
 * A _closure_ is a function with data attached.
 
In principle, the two can do similar things. Closures originate from functional programming languages that do not 
support OOP, where they serve as the poor man's object. Python is a multi-paradigm language, so it supports both. 

Objects are probably more common in Python, but there a situations where closures are idiomatic (such as the above).

# Callables

A similar effect can be achieved with callable objects: in Python, instances of any class can be made callable by adding a `__call__` method to the class:

In [8]:
class Integrand:
    def __init__(self, data):
        self.data = data
    def __call__(self, x):
        return exp(-x*sum(data))

The effect is that instances of the class can be called like a function:

In [9]:
data = [1, 2, 3, 4]
my_integrand_2 = Integrand(data) # an instance of Integrand
my_integrand_2(.1)

0.36787944117144233

We can use this the same way as the closure:

In [10]:
quad(my_integrand_2, 0, 5)[0]

0.10000000000000002

The technical term for a callable object is "functor". The built-in function `callable` can be used to determine whether any object or class is callable:

In [11]:
type(my_integrand_1) # closure

function

In [12]:
callable(my_integrand_1)

True

In [13]:
type(my_integrand_2) # functor = callable type

__main__.Integrand

In [14]:
callable(my_integrand_2)

True

In [15]:
type(my_integrand_2.__call__) # method

method

In [16]:
callable(my_integrand_2.__call__)

True

In [17]:
type(Integrand) # class

type

In [18]:
callable(Integrand)

True